# Pretrained Model
### The reason that it is introduced
* [ILSCVRC](http://image-net.org/)에서 1000개의 클래스를 classification하는 문제를 solving한 모델이 있다.
* 이모델은 1000개를 분류할 수 있는 능력이 있지만 수백만장의 이미지 데이터를 다시 트레이닝하는 것은 매우 무리다.
* 그래서 연구소에서는 이모델을 open source로 제공하는 방향으로 연구를 하고 이를 우리는 사용해 좋은 **visual features **를 이용할 수 있게 되었다.
* 사용법은 매우 간단하다.

In [1]:
import os
import random

import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

random_seed = 4332
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [2]:
transform_train = transforms.Compose(
    [transforms.RandomResizedCrop(224), # Data augmentation, 224 = image size, ImageNet pretrained model을 위해서 resize
     # 224여야만 제대로 동작할 정도로 사이즈에 민감하다.
     transforms.RandomHorizontalFlip(), # Data 수 늘어나는 효과. data augmentation
     transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ]) # -1 ~ 1. dataset의 mean, std을 이요해서 normalize 

transform_test = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                      download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                         shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                      download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                        shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## TO DO - Load and Define pretrained model and fine-tuning
- Use torchvision.models.resnet18
- Re-define the last fully connected layer for our task (ImageNet Dataset: 1000 classes, CIFAR10 Dataset: 10 classes, so set the last fully connected layer's out_features to 10) 

In [3]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

In [4]:
# resnet_cls = models.resnet18(pretrained=True)
class ResNet18(nn.Module):
    def __init__ (self):
        super(ResNet18, self).__init__()
        # TODO
        self.resnet = models.resnet18(pretrained=True)
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, 10) #원래는 1000개였는데 10개로 수정
        
    def forward(self, x):
#         import pdb; pdb.set_trace()
        x = self.resnet(x) # Batch_size x 10
        return x
    
# resnet_cls = models.resnet50(pretrained=True)
class ResNet50(nn.Module):
    def __init__ (self):
        super(ResNet50, self).__init__()
        # TODO
        self.resnet = models.resnet50(pretrained=True)
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, 10) #원래는 1000개였는데 10개로 수정
        
    def forward(self, x):
#         import pdb; pdb.set_trace()
        x = self.resnet(x) # Batch_size x 10
        return x
    
# print(models.resnet18(pretrained=True))        

## Load Model

In [5]:
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
net = ResNet50()

os.environ["CUDA_VISIBLE_DEVICES"] = '0, 1'
# net = nn.DataParallel(net, output_device=1)

net = torch.nn.DataParallel(net)
net = net.to(device)

## Training

In [ ]:
training_epochs = 350
learning_rate = 0.1

criterion = nn.CrossEntropyLoss() 
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum = 0.9)
net.train()

scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[150,250], gamma=0.1)

for epoch in range(training_epochs):        
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()

        outputs = net(inputs) 
        loss =  criterion(outputs, labels)
        loss.backward()
        
        optimizer.step()
    
    
    if epoch % 20 == 19:
        #test the model
        net.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for index, [data, label] in enumerate(testloader):
                data = data.to(device)
                label = label.to(device)

                output = net.forward(data)
                _, prediction_index = torch.max(output, 1)

                total += label.size(0)
                correct += (prediction_index == label).sum().float()

            if total == 0:
                total = 1
            print('[%d] loss: %.10f        accuracy: %.4f' % (epoch + 1, loss.item() / 100, correct/total))

        net.train()
    else:
        print('[%d] loss: %.10f' % (epoch + 1, loss.item() / 100))
        
    scheduler.step()
        
print('Finished Training')

[1] loss: 0.0192320406
[2] loss: 0.0180291522
[3] loss: 0.0172904432
[4] loss: 0.0129108691
[5] loss: 0.0147390926
[6] loss: 0.0113065410
[7] loss: 0.0110768497
[8] loss: 0.0095188916
[9] loss: 0.0143878508
[10] loss: 0.0097623265
[11] loss: 0.0101609862
[12] loss: 0.0079125744
[13] loss: 0.0094624293
[14] loss: 0.0085840857
[15] loss: 0.0066166592
[16] loss: 0.0061035764
[17] loss: 0.0082792598
[18] loss: 0.0069868147
[19] loss: 0.0052550012
[20] loss: 0.0084766132        accuracy: 0.7287
[21] loss: 0.0080730236
[22] loss: 0.0063084018
[23] loss: 0.0065913022
[24] loss: 0.0061514878
[25] loss: 0.0065690327
[26] loss: 0.0061592877
[27] loss: 0.0071458638
[28] loss: 0.0070414382
[29] loss: 0.0051870018
[30] loss: 0.0056201786
[31] loss: 0.0050966167
[32] loss: 0.0057684809
[33] loss: 0.0062142217
[34] loss: 0.0079315394
[35] loss: 0.0057713419
[36] loss: 0.0055502999
[37] loss: 0.0052231467
[38] loss: 0.0052203417
[39] loss: 0.0054522449
[40] loss: 0.0053571957        accuracy: 0.8106
[

## Test

In [ ]:
class_correct = np.zeros(10)
class_total = np.zeros(10)

net.eval()
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2f %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))
print('Mean Accuracy : ', 100*np.sum(class_correct)/np.sum(class_total))

## Visualization

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
# Get a batch of training data
dataiter = iter(testloader)

In [ ]:
inputs, labels = dataiter.next()
outputs = net(inputs.cuda())
_, predicted = torch.max(outputs.data, 1)
imshow(torchvision.utils.make_grid(inputs), title=[classes[x] for x in predicted])